# A2: Vector Semantics

Nikolai Ilinykh, Mehdi Ghanimifard, Wafia Adouane and Simon Dobnik


The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Write all your answers and the code in the appropriate boxes below.

In this lab we will look at how to build distributional semantic models from corpora and use semantic similarity captured by these models to do semantic tasks. We are also going to examine how different vector composition functions for vectors work in approximating semantic similarity of phrases when compared to human judgements.

This lab uses code from `dist_erk.py` which contains functions similar to those shown in the lecture. You can use either functions to solve these tasks.

In [1]:
# the following command simply imports all the methods from that code.
from dist_erk import *

## 1. Loading a corpus

To train a distributional model, we first need a sufficiently large collection of texts which contain different words used frequently enough in different contexts. Here we will use a section of the Wikipedia corpus `wikipedia.txt` stored in `wikipedia.zip`. This file has been borrowed from another lab by [Richard Johansson](http://www.cse.chalmers.se/~richajo/).

When unpacked, the file is 151mb, hence if you are using the MLT servers you should store it in a temporary folder outside your home and adjust the `corpus_dir` path below. It may already exist in `/srv/data/computational-semantics/`.

In [25]:
corpus_dir = r'C:/Users/annin/Computational_semantics/02-vector-semantics/wikipedia'

## 2. Building a model

Now you are ready to build the model.  
Using the methods from the code imported above build three word matrices with 1000 dimensions as follows:  

(i) with raw counts (saved to a variable `space_1k`);  
(ii) with PPMI (`ppmispace_1k`);  
(iii) with reduced dimensions SVD (`svdspace_1k`).  
For the latter use `svddim=5`. **[5 marks]**

Your task is to replace `...` with function calls. Functions are imported from `dist_erk.py`, and they similar to functions shown during the lecture.

In [26]:
numdims = 1000
svddim = 5

# Which words to use as targets and context words?
# We need to count the words and keep only the N most frequent ones
# Which function would you use here with which variable?
#preprocessed_corpus = preprocess(corpus)
#test_cooccurrences()
ktw = do_word_count(corpus_dir,numdims)
#print(ktw)

wi = make_word_index(ktw)
words_in_order = sorted(wi.keys(), key=lambda w:wi[w]) #is this correct??? why is it the same
#print(words_in_order)

# Create different spaces (the original matrix space, the ppmi space, the svd space)
# Which functions with which arguments would you use here?
print('create count matrices')
space_1k = make_space(corpus_dir,wi,numdims)
print('ppmi transform')
ppmispace_1k = ppmi_transform(space_1k,wi)
print('svd transform')
svdspace_1k = svd_transform(ppmispace_1k,numdims,svddim)
print('done.')

reading file wikipedia.txt
create count matrices
reading file wikipedia.txt


1145485it [00:45, 25073.62it/s]


ppmi transform
svd transform
done.


In [27]:
# now, to test the space, you can print vector representation for some words
print('house:', space_1k['house'])

house: [2551 3714 3104  567  962  627  443  185  311  189  131   28   93  169
   81  125  151  408  194   89   79   29  217  184   62   15   31   70
   10    1   41   21    1   31   37    1   30    5   25    7    3   20
   11    1   32   36    2    5   65    4    0   46    8   18   28    0
   20    7    8   16   10   40    0  175   10    2    7   19    1  174
   11    3    1    6    0    0    0   10    9   11    7   24    4    4
   14   23   58    7    0   10    2    3   10    6   18    6   13    3
   22    0    3    5    3    7   14    3   40   20   19   15    6    8
   23    4    5    1   19    0    3    1    0   14    0   14   53    7
    7   11    6    5    5    4   12    6   53    1    1  433    4    0
    5    7    7   12    1    1    3    4   17    8   16    1    2   31
    1   12   14    1   44    6   14    9   38    7    2    6    8    1
   10    6   10    1    9    7    9    4    3    0    9   11    3    2
    0    2   11   37    2    0    2    1    5    9   10   16    4    6

Oxford Advanced Dictionary has 185,000 words, hence 1,000 words is not representative. We trained a model with 10,000 words, and 50 dimensions on truncated SVD. All matrices are available in the folder `pretrained` of the `wikipedia.zip`file. These are `ktw_wikipediaktw.npy`, `raw_wikipediaktw.npy`, `ppmi_wikipediaktw.npy`, `svd50_wikipedia10k.npy`. Make sure they are in your path as we load them below.

In [29]:
import numpy as np

numdims = 10000
svddim = 50

print('Please wait...')
ktw_10k       = np.load('./wikipedia/pretrained/ktw_wikipediaktw.npy', allow_pickle=True)
space_10k     = np.load('./wikipedia/pretrained/raw_wikipediaktw.npy', allow_pickle=True).all()
ppmispace_10k = np.load('./wikipedia/pretrained/ppmi_wikipediaktw.npy', allow_pickle=True).all()
svdspace_10k  = np.load('./wikipedia/pretrained/svd50_wikipedia10k.npy', allow_pickle=True).all()
print('Done.')


Please wait...
Done.


In [57]:
# testing semantic space
print('house:', space_10k['house'])
print('house:', svdspace_10k['prosper']) #will not be found

house: [2554 3774 3105 ...    0    0    0]


KeyError: 'prosper'

## 3. Testing semantic similarity

The file `similarity_judgements.txt` contains 7,576 pairs of words and their lexical and visual similarities (based on the pictures) collected through crowd-sourcing using Mechanical Turk as described in [1]. The score range from 1 (highly dissimilar) to 5 (highly similar). Note: this is a different dataset from the phrase similarity dataset we discussed during the lecture [2]. You can find more details about how they were collected in the papers.

The following code will transform similarity scores into a Python-friendly format:

In [32]:
word_pairs = [] # test suit word pairs
semantic_similarity = [] 
visual_similarity = []
test_vocab = set()

for index, line in enumerate(open('similarity_judgements.txt')):
    data = line.strip().split('\t')
    if index > 0 and len(data) == 3:
        w1, w2 = tuple(data[0].split('#'))
        # Checks if both words from each pair exist in the word matrix.
        if w1 in ktw_10k and w2 in ktw_10k:
            word_pairs.append((w1, w2))
            test_vocab.update([w1, w2])
            semantic_similarity.append(float(data[1]))
            visual_similarity.append(float(data[2]))
        
print('number of available words to test:', len(test_vocab-(test_vocab-set(ktw))))
print('number of available word pairs to test:', len(word_pairs))
#print(word_pairs)
#print(ktw_10k)
#print(list(zip(word_pairs, visual_similarity, semantic_similarity)))

number of available words to test: 12
number of available word pairs to test: 774


We are going to test how the cosine similarity between vectors of each of the three spaces (normal space, ppmi, svd) compares with the human similarity judgements for the words in the similarity dataset. Which of the three spaces best approximates human judgements?

For comparison of several scores, we can use [Spearman correlation coefficient](https://en.wikipedia.org/wiki/Spearman's_rank_correlation_coefficient) which is implemented in `scipy.stats.spearmanr` [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html). The values of the Sperman correlation coefficient range from -1, 0 to 1, where 0 indicates no correlation, 1 perfect correaltion and -1 negative correlation. Hence, the greater the number the better the similarity scores align. The p values tells us if the coefficient is statistically significant. For this to be the case, it must be less than or equal to $< 0.05$.

Here is how you can calculate Pearson's correlation coefficient betweeen the scores of visual similarity and semantic similarity of the available words in the test suite:

In [56]:
from scipy import stats

rho, pval = stats.spearmanr(semantic_similarity, visual_similarity)
print("""Visual Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval)) 


Visual Similarity vs. Semantic Similarity:
rho     = 0.7122
p-value = 0.0000


Let's now calculate the cosine similarity scores of all word pairs in an ordered list using all three matrices. **[6 marks]**

In [55]:
from sklearn.metrics.pairwise import cosine_similarity

#   space_10k cosine scores:
word_pairs_arrays = []
for w1, w2 in word_pairs:
    if w1 in space_10k and w2 in space_10k:
       word_tuple = [space_10k[w1]],[space_10k[w2]]
    word_pairs_arrays.append(word_tuple)
raw_similarities = []
for w1, w2 in word_pairs_arrays:
    raw_similarities.append(cosine_similarity(w1,w2))
scores_space10k = []
for item in raw_similarities:
    scores_space10k.append(item[0][0]) 
print(f"space_10k cosine scores {scores_space10k}")

#   ppmispace cosine scores:
word_pairs_arrays = []
for w1, w2 in word_pairs:
    if w1 in ppmispace_10k and w2 in ppmispace_10k:
       word_tuple = [ppmispace_10k[w1]],[ppmispace_10k[w2]]
    word_pairs_arrays.append(word_tuple)
ppmi_similarities = []
for w1, w2 in word_pairs_arrays:
    ppmi_similarities.append(cosine_similarity(w1,w2))
scores_ppmi = []
for item in ppmi_similarities:
    scores_ppmi.append(item[0][0]) 
print(f"ppmispace_10k cosine scores {scores_ppmi}")

#   svd space cosine scores:
word_pairs_arrays = []
for w1, w2 in word_pairs:
    if w1 in svdspace_10k and w2 in svdspace_10k:
       word_tuple = [svdspace_10k[w1]],[svdspace_10k[w2]]
    word_pairs_arrays.append(word_tuple)
svd_similarities  = []
for w1, w2 in word_pairs_arrays:
    svd_similarities.append(cosine_similarity(w1,w2))
scores_svd = []
for item in svd_similarities:
    scores_svd.append(item[0][0]) 
print(f"svdspace_10k cosine scores {scores_svd}")


space_10k cosine scores [0.8225343344374065, 0.8872057973594759, 0.8186279531466685, 0.8641170631512164, 0.9209106962592174, 0.8365560468285074, 0.9105543047663365, 0.839648226650919, 0.7589366871548553, 0.8729671430291858, 0.9196105130424982, 0.7823406678252945, 0.9385814369441277, 0.6712567699328357, 0.8119647954565936, 0.7762348891086438, 0.875790927958858, 0.8334099693105823, 0.8498040003205739, 0.8010238123812325, 0.9329582307086745, 0.8685328881068072, 0.9016535452309238, 0.9034115624672077, 0.8595810413395489, 0.9095905271212125, 0.9196059672024545, 0.9264745546789073, 0.8940138022865456, 0.6035018071770325, 0.9590760140404796, 0.505869075889037, 0.7958814635183579, 0.8579450500210567, 0.7585974002217748, 0.8377279786803278, 0.9194949327646111, 0.8335758092457648, 0.9137889719993155, 0.8526736596298231, 0.7440511994695074, 0.6028483799106399, 0.621440124024626, 0.8268897334463412, 0.6990249439362441, 0.9037427727220718, 0.7557653066169757, 0.9437849074877116, 0.8207956180814521,

Calculate correlation coefficients between lists of similarity scores and the real semantic similarity scores from the experiment. The scores of what model best correlates them? Is this expected? **[6 marks]**

In [53]:
# space10k model vs. semantic similarity scores
rho1, pval1 = stats.spearmanr(semantic_similarity, scores_space10k)
print("""Semantic Similarity vs. Cos sim scores (space10k):
rho     = {:.4f}
p-value = {:.4f}""".format(rho1, pval1)) 

# ppmi model vs. semantic similarity scores
rho2, pval2 = stats.spearmanr(semantic_similarity, scores_ppmi)
print("""Semantic Similarity vs. Cos sim scores (PPMI):
rho     = {:.4f}
p-value = {:.4f}""".format(rho2, pval2)) 

# svd model vs. semantic similarity scores
rho3, pval3 = stats.spearmanr(semantic_similarity, scores_svd)
print("""Semantic Similarity vs. Cos sim scores (SVD):
rho     = {:.4f}
p-value = {:.4f}""".format(rho3, pval3)) 



Semantic Similarity vs. Cos sim scores (space10k):
rho     = 0.1522
p-value = 0.0000
Semantic Similarity vs. Cos sim scores (PPMI):
rho     = 0.4547
p-value = 0.0000
Semantic Similarity vs. Cos sim scores (SVD):
rho     = 0.4232
p-value = 0.0000


**Your answer should go here:**

Looks like the PPMI model's scores have the highest correlation with the semantic similarity scores from the experiment. PPMI is a good measure to use when looking into words co-occuring, which is in a way what we are doing here (i.e. "repair" - "car", "car" - "shop", "shop" - "bag" could all co-occur and also share semantic similarity). We also think the SVD space might be even too dense, causing some of the meanings to disappear and thus the correlation score is not very high. On the other hand, the count-based space (10k) is probably too sparse.

We can also calculate correlation coefficients between lists of cosine similarity scores and the real visual similarity scores from the experiment. Which similarity model best correlates with them? How do the correlation coefficients compare with those from the previous comparison - and can you speculate why do we get such results? **[7 marks]**

In [58]:
# space10k model vs. visual similarity scores
rho4, pval4 = stats.spearmanr(visual_similarity, scores_space10k)
print("""Visual Similarity vs. Cos sim scores (space10k):
rho     = {:.4f}
p-value = {:.4f}""".format(rho4, pval4)) 

# ppmi model vs. visual similarity scores
rho5, pval5 = stats.spearmanr(visual_similarity, scores_ppmi)
print("""Visual Similarity vs. Cos sim scores (PPMI):
rho     = {:.4f}
p-value = {:.4f}""".format(rho5, pval5)) 

# svd model vs. visual similarity scores
rho6, pval6 = stats.spearmanr(visual_similarity, scores_svd)
print("""Visual Similarity vs. Cos sim scores (SVD):
rho     = {:.4f}
p-value = {:.4f}""".format(rho6, pval6)) 


Visual Similarity vs. Cos sim scores (space10k):
rho     = 0.1212
p-value = 0.0007
Visual Similarity vs. Cos sim scores (PPMI):
rho     = 0.3838
p-value = 0.0000
Visual Similarity vs. Cos sim scores (SVD):
rho     = 0.3097
p-value = 0.0000


**Your answer should go here:**

It seems like PPMI model has the highest correlation also with the visual similarity scores. This is at least somewhat expected, as we can expect the relatively high correlation between the semantic and visual similarity scores (0.7122) to be mirrowed also in this score.

## 4. Operations on similarities

We can perform mathematical operations on vectors to derive meaning predictions.

For example, we can perform `king - man` and add the resulting vector to `woman` and we hope to get the vector for `queen`. What would be the result of `stockholm - sweden + denmark`? Why? **[3 marks]**

If you want to learn more about vector differences between words (and words in analogy relations), check this paper [4].

**Your answer should go here:**

The result would be Copenhagen because if you deduct Stockholm from Sweden you get the vector for capital city and if you add Denmark to that vector you get the vector for the capital of Denmark.
We are thus looking for the Danish version of the vector for Stockholm!

Here is some code that allows us to calculate such comparisons.

In [50]:
from scipy.spatial import distance

def normalize(vec):
    return vec / veclen(vec)

def find_similar_to(vec1, space):
    # vector similarity funciton
    #sim_fn = lambda a, b: 1-distance.euclidean(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.correlation(a, b)
    #sim_fn = lambda a, b: 1-distance.cityblock(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.chebyshev(normalize(a), normalize(b))
    #sim_fn = lambda a, b: np.dot(normalize(a), normalize(b))
    sim_fn = lambda a, b: 1-distance.cosine(a, b)

    sims = [
        (word2, sim_fn(vec1, space[word2]))
        for word2 in space.keys()
    ]
    return sorted(sims, key = lambda p:p[1], reverse=True)

Here is how you apply this code. Comment on the results you get. **[3 marks]**

In [51]:
short = normalize(svdspace_10k['short'])
light = normalize(svdspace_10k['light'])
long = normalize(svdspace_10k['long'])
heavy = normalize(svdspace_10k['heavy'])


find_similar_to(light - (heavy - long), svdspace_10k)[:10]


[('long', 0.8733111261346901),
 ('above', 0.8259671977311955),
 ('around', 0.8030776291120685),
 ('sun', 0.7692439111243973),
 ('just', 0.7678481974778111),
 ('wide', 0.767257431992253),
 ('each', 0.7665960260861158),
 ('circle', 0.7647746702909336),
 ('length', 0.7601066921319761),
 ('almost', 0.7542351860536628)]

**Your answer should go here:**

We expected to receive a vector corresponding to "short", as it is to light what long is to heavy. At least the words "above", "around", "length", and "wide" could be seen as semantically somewhat similar to "short". We found it a bit suprising that the most similar word on the list was "long". After some discussions we came to the conclusion that these particular word embeddings have some limitations.

Find 5 similar pairs of pairs of words and test them. Hint: google for `word analogies examples`. You can also construct analogies that are not only lexical but also express other relations such as grammatical relations, e.g. `see, saw, leave, ?` or analogies that are based on world knowledge as in `question-words.txt` from the [Google analogy dataset](http://download.tensorflow.org/data/questions-words.txt) described in [3]. Does the resulting vector similarity confirm your expectations? Remember you can only do this test if the words are contained in our vector space with 10,000 dimensions. **[10 marks]**

In [59]:
queen = normalize(svdspace_10k['queen'])
king = normalize(svdspace_10k['king'])
man = normalize(svdspace_10k['man'])
woman = normalize(svdspace_10k['woman'])
book = normalize(svdspace_10k['book'])
fiction = normalize(svdspace_10k['fiction'])
fire = normalize(svdspace_10k['fire'])
water = normalize(svdspace_10k['water'])
action = normalize(svdspace_10k['action'])
pen = normalize(svdspace_10k['pen'])

find_similar_to(woman - (king - man), svdspace_10k)[:10]
find_similar_to(book - fiction,svdspace_10k)[:10]
find_similar_to(fire + water ,svdspace_10k)[:10]
find_similar_to(action - pen ,svdspace_10k)[:10]

[('action', 0.5151249392718301),
 ('efforts', 0.4590420084786493),
 ('attacks', 0.458229114220977),
 ('violence', 0.44721686131031557),
 ('against', 0.4442320684289396),
 ('campaign', 0.43462302230282623),
 ('conflict', 0.4337237488315002),
 ('attack', 0.4308158906827424),
 ('threat', 0.42745929617870726),
 ('actions', 0.4243601588571876)]

## 5. Semantic composition and phrase similarity **[20 marks]**

In this task, we are going to examine at how the composed vectors of phrases by different semantic composition functions/models introduced in [2] correlate with human judgements of similarity between phrases. We will use the the dataset from this paper which is stored in `mitchell_lapata_acl08.txt`. If you are interested about furtehr details about this task also refer to this paper.

(i) Process the dataset. The dataset contains human judgemements of similarity between phrases recorded one per line. The first column indicates the id of a participant making a judgement (`participant`), the next column is `verb`, followed by `noun` and `landmark`. From these three columns we can construct phrases that were compared by human informants, namely `verb noun` vs `verb landmark`. The next column `input` indicates a similarity score a participant assigned to a pair of such phrases on a scale from 1 to 7 where 1 is lowest and 7 is highest. The last column `hilo` groups the phrases into two sets: phrases where we expect low and phrases where we expect high similarity scores. This is because we want to test our compositional functions on two tasks and examine whether a function is discriminative between them. Correlation between scores could also be due to other reasons than semantic similarity and hence good prediction on both tasks simultaneously shows that a function is truly discriminating the phrases using some semantic criteria.

For extracting information you can use the code from the lecture to start with. How to structure this data is up to you - a dictionary-like format would be a good choice. Remember that each example was judged by several participants and phrases will repeat in the dataset. Therefore, you have to collect all judgments for a particular set of phrases and average them. This will become useful in step (iii).

(ii) Compose the vectors of the extracted word pairs by testing different compositional functions. In the lecture we introduced simple additive, simple multiplicative and combined models (details are described in [2]). Your task is to take a pair of phrases, e.g. the first example in the dataset `stray thought` and `stray roam` and for each phrase compute a composition of the vectors of their words using these functions, using one function per experiment run. For each phrase you will get a single vector. You can encode the words with any vector space introduced earlier (standard space, ppmi or svd) but your code should be structured in a way that it will be easy to switch between them. Finally, take the resulting (composed) vectors of phrase pairs in the dataset and calculate a cosine similarity between them.

(iii) Now you have cosine similairity scores between vectors of phrases but how do they compare with the average human scores that you calculated from the individual judgements from the `input` column of the dataset for the same phrases? Calculate Spearman rank correlation coefficient between two lists of the scores both for the `high` and the `low` task . 

We use the Spearmank rank correlation coefficient (or Spearman's rho) rather than Peason's correlation coefficent because we cannot compare cosine scores with human judgements directly. Cosine is a constinuous measure and human judgements are expressed as ranks. Also, we cannot say if 0.28 to 1 is the same (or different) to 6 to 7 in the human scores.  The Spearman rank correlation coeffcient turns the scores for all examples within each group first to ranks and then these ranks are correlated (or approximated to a linear function). 

In the end you should get a table similar to the one below from the paper. What is the best compositional function from those that you evaluated with your vector spaces and why?

<img src="res.png" alt="drawing" width="500"/>


In [60]:
with open("./mitchell_lapata_acl08.txt") as f:
    text = f.readlines()

averages_dict = {}

for line in text[1:]:
    line = line.split()
    values_key = ((line[1], line[2]), (line[1], line[3]),line[5])
    value = int(line[4])
    count = 1
    if values_key in averages_dict:
        averages_dict[values_key][0] += value
        averages_dict[values_key][1] += 1
    else:
        averages_dict[values_key] = [value, count]
#print(averages_dict)
for key in averages_dict:
    sum_value = averages_dict[key][0]
    count = averages_dict[key][1]
    average = sum_value / count
    averages_dict[key] = [average]

print(averages_dict)



{(('stray', 'thought'), ('stray', 'roam'), 'low'): [5.294117647058823], (('stray', 'discussion'), ('stray', 'digress'), 'high'): [5.529411764705882], (('stray', 'eye'), ('stray', 'roam'), 'high'): [5.176470588235294], (('stray', 'child'), ('stray', 'digress'), 'low'): [3.0], (('throb', 'body'), ('throb', 'pulse'), 'high'): [5.029411764705882], (('throb', 'head'), ('throb', 'shudder'), 'low'): [2.3529411764705883], (('throb', 'voice'), ('throb', 'shudder'), 'low'): [2.8823529411764706], (('throb', 'vein'), ('throb', 'pulse'), 'high'): [6.205882352941177], (('chatter', 'machine'), ('chatter', 'click'), 'high'): [4.323529411764706], (('chatter', 'girl'), ('chatter', 'gabble'), 'high'): [4.764705882352941], (('chatter', 'child'), ('chatter', 'click'), 'low'): [2.176470588235294], (('chatter', 'tooth'), ('chatter', 'gabble'), 'low'): [2.176470588235294], (('rebound', 'shot'), ('rebound', 'ricochet'), 'high'): [5.735294117647059], (('rebound', 'ball'), ('rebound', 'rally'), 'low'): [2.5], ((

In [61]:
cosine_similarities = {}
for key,value in averages_dict.items():
    #print(key[1])
    try:
        #print(key)
        #print(key[0][0])
        vector1 = normalize(space_10k[key[0][0]]) 
        
        #print(vector1)
        #print(key[0][1])
        vector2 = normalize(space_10k[key[0][1]])
        #print(vector2)
        #print(key[1][1])
        vector3 = normalize(space_10k[key[1][1]])
        #print(vector3)
        if len(vector1) > 0 and len(vector2) > 0 and len(vector3) >0:
            vectors1=[vector1+vector2]
        #print(vectors1)
            vectors2= [vector1+vector3]
        #print (f' Vector 2 {vectors2}')
            #cosine_similarities.append(cosine_similarity(vectors1,vectors2))
            cosine_similarities[key] = cosine_similarity(vectors1,vectors2)

    except:
        pass
print(cosine_similarities)

{(('bow', 'butler'), ('bow', 'submit'), 'low'): array([[0.73848528]]), (('bow', 'company'), ('bow', 'submit'), 'high'): array([[0.81739913]]), (('boom', 'sale'), ('boom', 'thunder'), 'low'): array([[0.902861]]), (('boom', 'gun'), ('boom', 'thunder'), 'high'): array([[0.95037681]]), (('bow', 'head'), ('bow', 'submit'), 'low'): array([[0.80201642]]), (('bow', 'government'), ('bow', 'submit'), 'high'): array([[0.81736778]]), (('boom', 'noise'), ('boom', 'thunder'), 'high'): array([[0.96713142]]), (('boom', 'export'), ('boom', 'thunder'), 'low'): array([[0.95332669]])}


In [62]:
combination = {}
for key,value in averages_dict.items():
    for key_cos,value_cos in cosine_similarities.items():
        average_value = averages_dict[key_cos]
        combination[key_cos] = value_cos , average_value


low = {}
high = {}
for key,value in combination.items():
    if key[2] == "low":
        low[key] = value
    else:
        high[key] = value



In [64]:
values0 = []
values1 =[]
for key,value in high.items():
    values0.append(value[0].tolist())
    values1.append(value[1])
flat_values0 = []
for sublist in values0:
    for num in sublist:
        flat_values0.append(num)

res = stats.spearmanr(flat_values0, values1)
print("""cosine similarity vs. Participant average:
rho     = {:.4f}
p-value = {:.4f}""".format(res[0], res[1]))

values0 = []
values1 =[]
for key,value in low.items():
    values0.append(value[0].tolist())
    values1.append(value[1])

flat_values0 = []
for sublist in values0:
    for num in sublist:
        flat_values0.append(num)

res2 = stats.spearmanr(flat_values0, values1)
print("""cosine similarity vs. Participant average:
rho     = {:.4f}
p-value = {:.4f}""".format(res2[0], res2[1]))


cosine similarity vs. Participant average:
rho     = 0.8000
p-value = 0.2000
cosine similarity vs. Participant average:
rho     = -0.8000
p-value = 0.2000


**Any comments/thoughts should go here:**

# Literature

[1] C. Silberer and M. Lapata. Learning grounded meaning representations with autoencoders. In Proceedings of the 52nd Annual Meeting of the Association for Computational Linguistics, pages 721–732, Baltimore, Maryland, USA, June 23–25 2014 2014. Association for Computational Linguistics.  

[2] Mitchell, J., & Lapata, M. (2008). Vector-based Models of Semantic Composition. In Proceedings of ACL-08: HLT (pp. 236–244). Association for Computational Linguistics.
  
[3] T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.

[4] E. Vylomova, L. Rimell, T. Cohn, and T. Baldwin. Take and took, gaggle and goose, book and read: Evaluating the utility of vector differences for lexical relation learning. arXiv, arXiv:1509.01692 [cs.CL], 2015.

## Statement of contribution

Briefly state how many times you have met for discussions, who was present, to what degree each member contributed to the discussion and the final answers you are submitting.

We met once on Tuesday the 16th for 4-5 hours. Everyone was present and contributed equally on solving the assignment.
We worked further on the assignment on Thursday during and after the help session (for around 3 hours) and the contributions were equal.

## Marks

The assignment is marked on a 7-level scale where 4 is sufficient to complete the assignment; 5 is good solid work; 6 is excellent work, covers most of the assignment; and 7: creative work. 

This assignment has a total of 60 marks. These translate to grades as follows: 1 = 17% 2 = 34%, 3 = 50%, 4 = 67%, 5 = 75%, 6 = 84%, 7 = 92% where %s are interpreted as lower bounds to achieve that grade.